# 46-932, Simulation Methods for Option Pricing: Homework 4

*Author*: Jordan Giebas <br>
*Due Date*: Feb. 15, 2018

## Question 1: *Practice on Conditional Monte Carlo*

Given the following SDES,

$$ dS = rSdt + vSdW_1, \ \ \  dv^2 = \alpha v^2 dt + \psi v^2 dW_2 $$

Use an Euler discretization scheme to simulate the paths of the underlying. Do this for two cases, $(\alpha,\psi) = (.10,.10) ; (0.10, 1.0)$. The parameters are given as, <br>

<ul>
    <li> 
        $S_0 = K = 100$
    <li>
        $r=0.05$
    <li>
        $T = 1$
    <li>
        $v^2(0) = 0.04$
    <li>
        $N=50$
</ul>

### Part (a)
Use standard Monte Carlo to estimate the call price and provide standard errors for both cases.

In [1]:
# Import needed modules
import pandas as pd
import numpy as np
from numpy import exp, sqrt, log
import scipy
from scipy.stats import norm
%matplotlib inline

# Define problem parameters
S_0 = K = 100
r = 0.05
T = 1.0
X_0 = 0.04 # Let X = v^2, just keep this in mind as we go
N = 50.0
dt = T/N

In [2]:
def pr1a( alpha, psi ):
    
    # Construct volatility dataframe
    vol_df = pd.DataFrame(index=pd.RangeIndex(0,51), columns=pd.RangeIndex(1,10001))

    # t0 - v^2 = X = 0.04, populate across columns
    vol_df.loc[0,:] = 0.04

    # Iterate using Euler Scheme
    for i in range(1,len(vol_df.index)):
        Z = np.random.standard_normal(size=10000)
        vol_df.loc[i,:] = vol_df.loc[i-1,:] + alpha*vol_df.loc[i-1,:]*dt + psi*vol_df.loc[i-1,:]*np.sqrt(dt)*Z
    
    # Apply square root to every cell since we need v in underlying evolution
    vol_df = vol_df.applymap(np.sqrt)
    
    # Construct stock dataframe
    stock_df = pd.DataFrame(index=pd.RangeIndex(0,51), columns=pd.RangeIndex(1,10001))

    # t0 - S_0 = 100
    stock_df.loc[0,:] = S_0

    # Iterate using Euler Scheme
    for i in range(1,len(stock_df.index)):

        Z = np.random.standard_normal(size=10000)
        stock_df.loc[i,:] = stock_df.loc[i-1,:] + r*stock_df.loc[i-1,:]*dt + vol_df.loc[i-1,:]*stock_df.loc[i-1,:]*np.sqrt(dt)*Z

    # Discount the payoffs
    d_payoffs = np.exp(-r*T)*np.maximum(np.array(stock_df.loc[50,:]) - K,0)
    print("Estimated Option Price: $%f \nStandard Error: %f" % (np.mean(d_payoffs),np.std(d_payoffs)/np.sqrt(10000)))

In [3]:
# Case I
print("==== CASE I ====")
pr1a(0.10,0.10)

# Case II
print("==== CASE II ====")
pr1a(0.10,1.0)

==== CASE I ====
Estimated Option Price: $10.612523 
Standard Error: 0.150179
==== CASE II ====
Estimated Option Price: $10.477421 
Standard Error: 0.153882


### Part (b)
Use conditional Monte Carlo. I.e., simulate the volatility paths and substitute $\sigma$ with 

$$\sqrt{\frac{1}{N}\sum_{i=1}^{N}\sigma^2(i\Delta)}$$

in the Black-Scholes formula to yield the price results. 

In [4]:
# Construct volatility dataframe
vol_df = pd.DataFrame(index=pd.RangeIndex(0,52), columns=pd.RangeIndex(1,10001))

# t0 - v^2 = X = 0.04, populate across columns
vol_df.loc[0,:] = 0.04

# Iterate using Euler Scheme
for i in range(1,len(vol_df.index)-1):
    Z = np.random.standard_normal(size=10000)
    vol_df.loc[i,:] = vol_df.loc[i-1,:] + alpha*vol_df.loc[i-1,:]*dt + psi*vol_df.loc[i-1,:]*np.sqrt(dt)*Z
    
# Store squared sum as in 51st row
vol_df.loc[51,:] = np.square(vol_df.loc[1:50,:]).sum()

# Get only the first 50 columns
tmp = vol_df.loc[1:50,:]
tmp = tmp.applymap(np.square)
vol_df.loc[51,:] = [tmp[i].sum() for i in range(1,10001)]

NameError: name 'alpha' is not defined

In [5]:
# Define necessary functions for BS (with dividends) price
def d_plus( x, K, tau, sigma, rfr, q ):
    
    return ( (log(x/K) + ((rfr - q + 0.5*(sigma**2))*tau))/(sigma*sqrt(tau)) )

def d_minus( x, K, tau, sigma, rfr, q ):
    
    return ( d_plus(x, K, tau, sigma, rfr, q) - sigma*sqrt(tau) )

def BS_price( x, K, r, sigma, tau, q ):
    
    d_1 = d_plus(x, K, tau, sigma, r, q)
    d_2 = d_minus(x, K, tau, sigma, r, q)
    
    return ( x*exp(-1.0*q*tau)*norm.cdf(d_1) - K*exp(-1.0*r*tau)*norm.cdf(d_2) )

In [6]:
bs_prices = [BS_price(S_0,K,r,vol_df.loc[51,j],T,0) for j in range(1,10001)]

In [7]:
print("Conditional Monte Carlo Estimated Call Price: ", np.mean(bs_prices))
print("Associated Standard Error: ", np.std(bs_prices)/sqrt(10000))

Conditional Monte Carlo Estimated Call Price:  nan
Associated Standard Error:  nan


There is a variance reduction but we're off in the actual price of the call. 

### Question 2: *Practice on interest rate derivatives and CIR*

Consider the CIR square root diffusion spot-rate model,

$$ dr(t) = \alpha(b-r(t))dt + \sigma\sqrt{r(t)}dW(t) $$

where $t \in [0,T]$. The parameteres are given below:

<ul>
    <li> $\alpha = 0.2$
    <li> $\sigma = 0.1$
    <li> $b = 0.05$
    <li> $r(0) = 0.04$
    <li> $N = 50$
    <li> $n = 1000$
</ul>

Where $n,N$ are the number of paths and time steps respectively.

### Part (a)

Find the price of a zero coupon bond paying \$1 at time $T=1$. 

In [125]:
## Define pr2 parameters
alpha = 0.2
sigma = 0.1
b = 0.05
r_0 = 0.04
n = 1000
N = 50
T = 1
dt = T/N

# Constant model parameter
nu = (4*b*alpha)/(sigma**2)

In [126]:
# Construct the interest rate process dataframe
r_df = pd.DataFrame(index=pd.RangeIndex(0,52), columns=pd.RangeIndex(1,1001))
r_df.iloc[0,:] = r_0

In [127]:
# Since d > 1, we use Case I referencing Glasserman pg. 124
for i in range(1,len(r_df)-1):
    
    ## Dynamic model parameters, since t-s = dt iterating over each time step
    const = ((1-np.exp(-alpha*dt))*(sigma**2))/(4*alpha)
    lam = r_df.loc[i-1,:]*(np.exp(-alpha*dt)/const)
    
    # Generate Standard Normal, non-central Chi-squared
    #Z = np.random.standard_normal(size=1000)
    X = np.random.noncentral_chisquare(df=nu,nonc=lam.tolist(),size=1000)
    #X = np.random.chisquare(df=nu,size=1000)
    
    #r_df.loc[i,:] = const*( (Z + np.sqrt(lam.tolist()))**2 + X )
    r_df.loc[i,:] = const*X

In [128]:
r_df.loc[51,:] = r_df.loc[0:50,:].sum()

In [129]:
arg = -1.0*dt*np.array(r_df.loc[51,:])

In [130]:
c_i = np.exp(arg.tolist())

In [131]:
np.mean(c_i)

0.95958254137905474

In [132]:
np.std(c_i)/np.sqrt(1000)

0.00032415368652886792

The above suggests the price of the bond is \$0.9585 with a standard error of 0.00033.

### Part (b)
I don't understand the question.

## Question 3: *Replicating Glasserman's 'Greeks' methodology*

Replicate the Delta and Vega values found in Table 1 on pg. 276 of the Glasserman and Broadie text: report the value and associated standard errors. Perform this replication using the resimulation, pathwise, and likelihood ratio methods. Use the following parameters:

<ul>
    <li>$r=0.10$</li>
    <li>$K=100$</li>
    <li>$\delta = 0.03$</li>
    <li>$\sigma = 0.25$</li>
    <li>$T=0.20$</li>
    <li>$n=10000$</li>
    <li>$h=0.0001$</li>
</ul>

Use the formulas in the Glassman text to simulate the Greeks, and $S_{T}$ as a control variable during the simulations.

In [31]:
# Define the global parameters
r = 0.10
K = 100
q = 0.03
sig = 0.25
T = 0.20
n = 10000
h = 0.0001
S_0 = 90

In [3]:
# Generate price process df
pp_df = pd.DataFrame(index=pd.RangeIndex(start=0,stop=2001),columns=pd.RangeIndex(start=1, stop=10001))
pp_df.loc[0,:] = S_0

In [28]:
for i in range(1,len(pp_df)-1):
    Z = np.random.standard_normal(size=10000)
    pp_df.loc[i,:] = pp_df.loc[i-1,:]*np.exp( (r-q-0.5*(sig**2))*(h) + sig*np.sqrt(h)*Z )

In [15]:
# Generate delta df using central differences
pp_ph_df = pp_df + h
pp_mh_df = pp_df - h

In [32]:
pp_df.loc[2000,:] = np.maximum(pp_df.loc[1999,:] - K, 0)

In [35]:
indicator = pp_df.loc[2000,:]

In [58]:
vals = (np.exp(-r*T)/sig)*(np.log(np.divide(pp_df.loc[1999,:],pp_df.loc[0,:]).tolist()) - (r-q+0.5*sig**2)*T)*indicator

In [44]:
x = np.array([1,2,3])
y = np.array([2,3,4])
z = np.divide(x,y)

In [59]:
np.mean(vals)

0.92371318740192621

In [56]:
# Closed form to check
vega = S_0*np.sqrt(T)*scipy.stats.norm.cdf((log(S_0/K) + ((r - q + 0.5*(sig**2))*T))/(sig*sqrt(T)))

In [57]:
vega

8.9857761495686397